In [10]:
#from DeepPurpose import models
from DeepPurpose.utils import *
from DeepPurpose.dataset import *
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs

In [11]:
# protein_dataset = pd.read_pickle("data/ecoli_string_structure_df.pkl")
# davis = pd.read_pickle("data/davis_processed.pkl")
# bindingdb = pd.read_pickle("data/bindingdb_processed.pkl")
# combined = pd.read_pickle("data/combined_processed.pkl")
# pmi = pd.read_csv("data/pmi_processed.csv", header=0)
# pmicat = pd.read_csv("data/pmi_processed_cat.csv", header=0)
# ecoli_df = pd.read_pickle("data/ecoli_string_structure_df.pkl")
ecoli_1430 = pd.read_csv("data/proteins-1430-dp.csv",header=0)

In [15]:
target_encoding_map = ecoli_1430
target_encoding_map

,Protein,Sequence
0,A0A024FRL9,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...
1,A0A024L327,MTLSFITRWRDELPATYTTLSPTPLNNARLIWHNAELANTLGIPSS...
2,A0A061LQM0,MTQSAMCIPLWPARNGNTAHLVMCPFAGGSSSAFRHWQAEQLADCA...
3,A0A076YIY5,MGFLRSGCYPDSCHADWYRYNLTYRQCKSGPPMAGRFVWKPDLMFK...
4,A0A080FXI0,MNKVGMFYTYWSTEWMVDFPATAKRIAGLGFDLMEISLGEFHNLSD...
...,...,...
1420,Q9X2V7,MTVPISIIFWGNIMKKHLVVIAFCVLFASASAFAAKGTDSLKSSIE...
1421,Q9X4B7,MAKELSVYGPTAGESMGGTGANLNQQGGNNNSNSGVHWGGGSGSGN...
1422,Q9X4G2,MNKVYNTVWNESTGTWVVTSELTRKGGLRPRQIKRTVLAGLIAGLL...
1423,Q9X6W1,MAKHLFTSESVSEGHPDKIADQISDAVLDAILEQDPKARVACETYV...


In [21]:
uni_target = target_encoding_map['Sequence']
uni_target

0       MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...
1       MTLSFITRWRDELPATYTTLSPTPLNNARLIWHNAELANTLGIPSS...
2       MTQSAMCIPLWPARNGNTAHLVMCPFAGGSSSAFRHWQAEQLADCA...
3       MGFLRSGCYPDSCHADWYRYNLTYRQCKSGPPMAGRFVWKPDLMFK...
4       MNKVGMFYTYWSTEWMVDFPATAKRIAGLGFDLMEISLGEFHNLSD...
                              ...                        
1420    MTVPISIIFWGNIMKKHLVVIAFCVLFASASAFAAKGTDSLKSSIE...
1421    MAKELSVYGPTAGESMGGTGANLNQQGGNNNSNSGVHWGGGSGSGN...
1422    MNKVYNTVWNESTGTWVVTSELTRKGGLRPRQIKRTVLAGLIAGLL...
1423    MAKHLFTSESVSEGHPDKIADQISDAVLDAILEQDPKARVACETYV...
1424    MTTQVPPSALLPLNPEQLARLQAATTDLTPTQLAWVSGYFWGVLNQ...
Name: Sequence, Length: 1425, dtype: object

In [18]:
# This step creates pseudo drug and label to fit in deep purpose
psedosmiles=np.repeat("",len(uni_target))
pseudoy=np.ones(len(uni_target))
len(psedosmiles)

1425

In [22]:
# define your target encoding and drug encoding here
# ususally we use PseodoAAC for drug encoding, but that takes some time to generate
# if the target dataset is large, you can split it into smaller files and generate one by one
# keep in mind that drug is pseudo
drug_encoding, target_encoding = 'Pubchem', 'Transformer'
train, val, test = data_process(psedosmiles, uni_target, pseudoy,
                                drug_encoding, target_encoding,
                                split_method='random',
                                frac=[1,0,0])

Drug Target Interaction Prediction Mode...
in total: 1425 drug-target pairs
encoding drug...
unique drugs: 1
encoding protein...
unique target sequence: 1425
splitting dataset...
Done.


In [23]:
train

,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([1, 637, 210, 3576, 3662, 238, 27, 1282, 93, ..."
1,,MTLSFITRWRDELPATYTTLSPTPLNNARLIWHNAELANTLGIPSS...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([1148, 252, 79, 372, 119, 637, 59, 272, 35, 1..."
2,,MTQSAMCIPLWPARNGNTAHLVMCPFAGGSSSAFRHWQAEQLADCA...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([226, 129, 118, 436, 47, 485, 48, 81, 222, 48..."
3,,MGFLRSGCYPDSCHADWYRYNLTYRQCKSGPPMAGRFVWKPDLMFK...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([132, 56, 735, 4, 198, 106, 496, 49, 554, 174..."
4,,MNKVGMFYTYWSTEWMVDFPATAKRIAGLGFDLMEISLGEFHNLSD...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([202, 825, 3936, 182, 308, 96, 867, 62, 178, ..."
...,...,...,...,...,...
1420,,MTVPISIIFWGNIMKKHLVVIAFCVLFASASAFAAKGTDSLKSSIE...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([226, 90, 71, 122, 2770, 210, 839, 91, 42, 11..."
1421,,MAKELSVYGPTAGESMGGTGANLNQQGGNNNSNSGVHWGGGSGSGN...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([756, 30, 290, 108, 231, 41, 189, 1017, 52, 3..."
1422,,MNKVYNTVWNESTGTWVVTSELTRKGGLRPRQIKRTVLAGLIAGLL...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([202, 1432, 2971, 23, 1557, 52, 347, 42, 94, ..."
1423,,MAKHLFTSESVSEGHPDKIADQISDAVLDAILEQDPKARVACETYV...,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","([756, 91, 195, 843, 3727, 1336, 3746, 13, 835..."


In [27]:
target_enc=train.drop(columns=["SMILES","Label","drug_encoding"])
target_enc = target_enc.rename(columns={"target_encoding": target_encoding})
target_enc

,Target Sequence,Transformer
0,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...,"([1, 637, 210, 3576, 3662, 238, 27, 1282, 93, ..."
1,MTLSFITRWRDELPATYTTLSPTPLNNARLIWHNAELANTLGIPSS...,"([1148, 252, 79, 372, 119, 637, 59, 272, 35, 1..."
2,MTQSAMCIPLWPARNGNTAHLVMCPFAGGSSSAFRHWQAEQLADCA...,"([226, 129, 118, 436, 47, 485, 48, 81, 222, 48..."
3,MGFLRSGCYPDSCHADWYRYNLTYRQCKSGPPMAGRFVWKPDLMFK...,"([132, 56, 735, 4, 198, 106, 496, 49, 554, 174..."
4,MNKVGMFYTYWSTEWMVDFPATAKRIAGLGFDLMEISLGEFHNLSD...,"([202, 825, 3936, 182, 308, 96, 867, 62, 178, ..."
...,...,...
1420,MTVPISIIFWGNIMKKHLVVIAFCVLFASASAFAAKGTDSLKSSIE...,"([226, 90, 71, 122, 2770, 210, 839, 91, 42, 11..."
1421,MAKELSVYGPTAGESMGGTGANLNQQGGNNNSNSGVHWGGGSGSGN...,"([756, 30, 290, 108, 231, 41, 189, 1017, 52, 3..."
1422,MNKVYNTVWNESTGTWVVTSELTRKGGLRPRQIKRTVLAGLIAGLL...,"([202, 1432, 2971, 23, 1557, 52, 347, 42, 94, ..."
1423,MAKHLFTSESVSEGHPDKIADQISDAVLDAILEQDPKARVACETYV...,"([756, 91, 195, 843, 3727, 1336, 3746, 13, 835..."
